In [1]:
# Aligning prompts/responses with Kaggle agents
# Removing some errors, fixing incorrect formatting, etc.
# Removing reasoning part from the responses

import pickle

In [89]:
# d_b = pickle.load(open('dataset_11194_basic.pkl', 'rb'))
d_b = pickle.load(open('!final_dataset_99844_with_hints_after_n_mini_full.pkl', 'rb'))

In [102]:
set([o['type'] for o in d_b])

{'answer', 'guess', 'question_pick', 'question_pick_with_answer'}

In [101]:
set([o['response'] for o in d_b if o['type'] == "answer"])

{'no', 'yes'}

In [92]:
for o in d_b:
    o['prompt'] = o['prompt'].replace("given this reasoning i will guess", "").replace("i will guess", "").replace("Given this reasoning, the best next question would be: ", "").replace("Given this reasoning, the best next question is: ", "").replace("Given this reasoning, I will ask: ", "").replace("Given this reasoning, the best next question to ask is: ", "").replace("The question that best fits this reasoning is: ", "").replace("I will guess: ", "")
    o['response'] = o['response'].replace("given this reasoning i will guess", "").replace("i will guess", "").replace("Given this reasoning, the best next question would be: ", "").replace("Given this reasoning, the best next question is: ", "").replace("Given this reasoning, I will ask: ", "").replace("Given this reasoning, the best next question to ask is: ", "").replace("The question that best fits this reasoning is: ", "").replace("I will guess: ", "")

In [93]:
for o in d_b:
    if o['type'] != "answer":
        continue
    o['prompt'] = o['prompt'].replace("""Format your answer like this: First, reason about your answer for 1-3 sentences. If the answer is obvious - state it. If not - reason about whether YES or NO would be more helpful and truthful.
Then, your last line should be the answer, only YES or NO.
Don't output anything else.""", """Format your answer like this: Your last and only line should be the answer, only YES or NO.
Don't output anything else.""")
    
    if "reason" in o['prompt'].lower() and not "Does the keyword relate to reasoning" in o['prompt']:
        print("!!!")
        break
    o['response'] = o['response'].split("\n")[-1].strip().replace("Answer: ", "").replace(".", "").lower()
    if o['response'].lower() not in ['yes', 'no']:
        print("!", o['response'])

In [94]:
for o in d_b:
    if o['type'] != "guess":
        continue
    o['prompt'] = o['prompt'].replace("""Format your answer like this: First, reason about your decision for 2-4 sentences. Summarize what you know, summarize space of possibilities, etc.
Then, your last line should be the keyword that you want to guess.
Don't output anything else. Only output the keyword that you want to guess. Don't say anything like "My guess is {keyword}" - always just "{keyword}\"""", 
              """Format your answer like this: only output the keyword that you want to guess, nothing else. Don't say anything like "My guess is {{keyword}}" - always just "{{keyword}}\"""")
    if "sentences" in o['prompt'].lower():
        print("!!!")
        break
    o['response'] = o['response'].split("\n")[-1].strip().replace("Answer: ", "").replace("Is it an ", "").replace("Is it a ", "").replace("Is it in", "").replace("?", "").replace("\"", "").replace("I will guess ", "")
    # OSB (Oriented Strand Board). - clean anything inside parentheses using re
    import re
    o['response'] = re.sub(r'[^a-zA-Z\s]', '', re.sub(r'\([^)]*\)', '', o['response'])).strip()
    # if any symbol is not alphabetical  -output the response
    if any([not c.isalnum() and c != " " and c != "-" and c != "." and c != "," and c != "'" for c in o['response']]):
        if o['response'] == "M&M's" or "sorry" in o['response'].lower():
            continue
        print("!", o['response'])
        break

In [95]:
print(o['prompt'])

You are playing "20 Questions" game.
    
Here are the previous questions and answers:
    Question: Is the keyword a living thing? Answer: no Your guess: chair
Question: Is the keyword something commonly found indoors? Answer: no Your guess: tree
Question: Is the keyword a man-made object? Answer: no Your guess:  rock
Question: Is the keyword something commonly found outdoors? Answer: yes Your guess: mountain
Question: Is the keyword larger than a car? Answer: yes Your guess: hill
Question: Is the keyword something that can be classified as a geological feature? Answer: no Your guess:  ocean
Question: Is the keyword a type of natural formation, like a rock or a tree? Answer: no Your guess:  lake
Question: Is the keyword a type of landform, like a mountain or hill? Answer: no Your guess: river
Question: Is the keyword a type of body of water, like a lake or river? Answer: no Your guess: mountain range
Question: Is the keyword something that is often displayed in museums? Answer: yes Yo

In [96]:
d_b = [o for o in d_b if (o['type'] != "question_pick") or ("?" in o['response'])]
for o in d_b:
    if o['type'] != "question_pick" and o['type'] != "question_pick_with_answer":
        continue
    o['prompt'] = o['prompt'].replace("""You goal is to pick the best next question.
Safe questions are usually ones that split the space of possibilities in half given the previous answers.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.

In any case, try to make reason about what decision is the best here, balance risk, and choose the best next question. Act as a competetive extremely smart player that you are.

Don't forget that some inconsistencies in answers can be caused by errors in the answering LLM.



Format your answer like this: First, reason about your decision for 2-4 sentences. Summarize what you know, summarize space of possibilities, reason about what question would be the most beneficial.
Then, your last line should be the questions that you've picked.
Don't output anything else""", """You goal is to pick the best next question.
Safe questions are usually ones that split the space of possibilities in half given the previous answers.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.

In any case, try to make reason about what decision is the best here, balance risk, and choose the best next question. Act as a competetive extremely smart player that you are.

Don't forget that some inconsistencies in answers can be caused by errors in the answering LLM.

Format your answer like this: First, reason about your decision for 2-4 sentences. Summarize what you know, summarize space of possibilities, reason about what question would be the most beneficial.
Then, your last line should be the questions that you've picked.
Don't output anything else""")
    o['prompt'] = o['prompt'].replace("""You goal is to pick the best next question.
Safe questions are usually ones that split the space of possibilities in half given the previous answers.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.

In any case, try to make reason about what decision is the best here, balance risk, and choose the best next question. Act as a competetive extremely smart player that you are.

Don't forget that some inconsistencies in answers can be caused by errors in the answering LLM.

Format your answer like this: First, reason about your decision for 2-4 sentences. Summarize what you know, summarize space of possibilities, reason about what question would be the most beneficial.
Then, your last line should be the questions that you've picked.
Don't output anything else.""", 
                          """Your goal is to choose the best next question.
Safe questions are usually ones that split the space of possibilities in half given the previous answers.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.

You should keep balance of safe questions and more creative and non-orthodox ones.
Safe questions are usually ones that split the space of possibilities in half.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.
Consider asking non-standard questions like "Does the keyword consist of a single word?, however only do it if you don't know it yet, and if you think that it's important to know at this point in the game.
Consider asking questions like "Does the keyword start with letter 'A'?" or "Does the keyword contain letter 'E'?". However, don't overdo it, focus on more substantial questions.

In any case, try to make reason about what decision is the best here, balance risk, and choose the best next question. Act as a competetive extremely smart player that you are.

Don't forget that some inconsistencies in answers can be caused by errors in the answering LLM.

Format your answer like this: Your last and only line should be the question that you want to ask. Just output the question, not "Question: ..." or "I pick question..."
Don't output anything else!""")
    if "sentences" in o['prompt']:
        print("!!!")
        break
    o['response'] = o['response'].split("\n")[-1].strip().split(":")[-1].replace("I will ask ", "")
    o['response'] = o['response'].replace("if it is commonly used for storing large amounts of data.", "Is the keyword commonly used for storing large amounts of data?")
    if "?" not in o['response'] or ":" in o['response'] or "my" in o['response'].lower() or "i " in o['response'].lower()  or "here " in o['response'].lower():
        if "astronomy" in o['response'].lower() or "atmosphere" in o['response'].lower() or "mythology" in o['response'].lower() or "creamy" in o['response'].lower() or "Myrtaceae" in o['response']:
            continue
        if "where" in o['response'].lower() or "chili " in o['response'].lower() or "MIDI " in o['response'] or "anatomy" in o['response'].lower() or "gummy" in o['response'].lower() or "mycorrhizal" in o['response'].lower():
            continue
        print("!", o['response'])
        break

In [97]:
# print(o['prompt'])

In [98]:
d_b[50000]['game_state']

Game(keyword='spare bulb', rounds=[Round(question='Is the keyword a living thing?', answer='no', guess='chair'), Round(question='Is the keyword a physical object?', answer='yes', guess='table'), Round(question='Is the keyword something commonly found indoors?', answer='yes', guess='sofa'), Round(question='Is the keyword typically found in a kitchen?', answer='no', guess='lamp'), Round(question='Is the keyword something commonly found in a living room?', answer='no', guess='i will guess bookshelf'), Round(question='Is the keyword something commonly found in a bathroom?', answer='no', guess='given this reasoning i will guess desk'), Round(question='Is the keyword typically found in an office?', answer='no', guess='toaster'), Round(question='Is the keyword something commonly found in a garage?', answer='no', guess='bed')], win=None)

In [99]:
d_b = [o for o in d_b if not o['type'] == "questions_candidates" and not o['type'] == "questions_candidates_with_answer"]

In [100]:
pickle.dump(d_b, open('!final_dataset_99844_with_hints_after_n_mini_full.pkl', 'wb'))

In [103]:
len(d_b)

74883

In [65]:
# d_b = pickle.load(open("dataset_6016_with_hints_after_n_step.pkl", 'rb'))
d_b = pickle.load(open("!final_dataset_6016_with_hints_after_n_step.pkl", 'rb'))

In [66]:
set([o['type'] for o in d_b])

{'answer', 'guess', 'question_pick', 'question_pick_with_answer'}

In [22]:
for o in d_b:
    if o['type'] != "answer":
        continue
    o['prompt'] = o['prompt'].replace("""Format your answer like this: First, reason about your answer for 1-3 sentences. If the answer is obvious - state it. If not - reason about whether YES or NO would be more helpful and truthful.
Then, your last line should be the answer, only YES or NO.
Don't output anything else.""", """Format your answer like this: Your last and only line should be the answer, only YES or NO.
Don't output anything else.""")
    if "reason" in o['prompt'].lower():
        print("!!!")
        break
    o['response'] = o['response'].split("\n")[-1].strip().replace("Answer: ", "").lower()
    if o['response'].lower() not in ['yes', 'no']:
        print("!", o['response'])
        break
    # break

In [24]:
for o in d_b:
    if o['type'] != "guess":
        continue
    o['prompt'] = o['prompt'].replace("""Format your answer like this: First, reason about your decision for 2-4 sentences. Summarize what you know, summarize space of possibilities, etc.
Then, your last line should be the keywords that you want to guess.
Don't output anything else. Only output the keywords that you want to guess. Don't say anything like "My guess is {keyword}" - always just "{keyword}\"""", 
              """Format your answer like this: only output the keyword that you want to guess, nothing else. Don't say anything like "My guess is {{keyword}}" - always just "{{keyword}}\"""")
    if "sentences" in o['prompt'].lower():
        print("!!!")
    o['response'] = o['response'].split("\n")[-1].strip().replace("Answer: ", "").replace("Is it an ", "").replace("Is it a ", "").replace("Is it in", "").replace("?", "").replace("\"", "")
    # if any symbol is not alphabetical  -output the response
    if any([not c.isalnum() and c != " " and c != "-" and c != "." and c != "," and c != "'" for c in o['response']]):
        print("!", o['response'])

! Aoraki / Mount Cook
! Aoraki / Mount Cook


In [88]:
d_b = [o for o in d_b if (o['type'] != "question_pick" and o['type'] != "question_pick_with_answer") or ("?" in o['response'])]
for o in d_b:
    if o['type'] != "question_pick" and o['type'] != "question_pick_with_answer":
        continue
    o['prompt'] = o['prompt'].replace("""Your goal is to pick the best next question.
Safe questions are usually ones that split the space of possibilities in half given the previous answers.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.

In any case, try to make reason about what decision is the best here, balance risk, and choose the best next question. Act as a competetive extremely smart player that you are.

Don't forget that some inconsistencies in answers can be caused by errors in the answering LLM.""", "")
    o['prompt'] = o['prompt'].replace("""Format your answer like this: First, reason about your decision for 2-4 sentences. Summarize what you know, summarize space of possibilities, reason about what question would be the most beneficial.
Then, your last line should be the questions that you've picked.
Don't output anything else.""", 
                          """You goal is to choose the best next question.
Safe questions are usually ones that split the space of possibilities in half given the previous answers.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.

You should keep balance of safe questions and more creative and non-orthodox ones.
Safe questions are usually ones that split the space of possibilities in half.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.
Consider asking non-standard questions like "Does the keyword consist of a single word?", however only do it if you don't know it yet, and if you think that it's important to know at this point in the game.
Consider asking questions like "Does the keyword start with letter 'A'?" or "Does the keyword contain letter 'E'?". However, don't overdo it, focus on more substantial questions.

In any case, try to make reason about what decision is the best here, balance risk, and choose the best next question. Act as a competitive extremely smart player that you are.

Don't forget that some inconsistencies in answers can be caused by errors in the answering LLM.

Format your answer like this: Your last and only line should be the question that you want to ask. Just output the question, not "Question: ..." or "I pick question..."
Don't output anything else!""")
    if "sentences" in o['prompt']:
        print("!!!")
        break
    o['response'] = o['response'].split("\n")[-1].strip().split(":")[-1]
    if "?" not in o['response'] or ":" in o['response'] or "my" in o['response'].lower() or "i " in o['response'].lower()  or "here " in o['response'].lower():
        print("!", o['response'])

! Is it associated with a legend or myth?
! Is the keyword a Māori name?
! Is it commonly found in places where food is served?


In [108]:
print([o for o in d_b if o['type'] == "question_pick_with_answer"][300]['prompt'])

You are playing "20 Questions" game.
    
Here are the previous questions and answers:
    Question: Is it a thing? Answer: yes
Question: Is it man-made? Answer: yes
Question: Is it used in everyday life? Answer: yes
Question: Is it electronic? Answer: no
Question: Is it made of metal? Answer: no
Question: Is it made of plastic? Answer: no
Question: Is it made of wood? Answer: no
Question: The best next question would be: Is it used for personal hygiene? Answer: no
Question: Is it used in the kitchen? Answer: no
Question: Is it made of fabric? Answer: no
Question: Is it made of paper? Answer: yes
Question: Is it related to writing or drawing? Answer: yes
Question: Is it used in an office setting? Answer: yes
Question: Is it used for documentation purposes? Answer: yes

    




You goal is to choose the best next question.
Safe questions are usually ones that split the space of possibilities in half given the previous answers.
If you have a few questions left, you can start acting more

In [26]:
d_b = [o for o in d_b if not o['type'] == "questions_candidates" and not o['type'] == "questions_candidates_with_answer"]

In [104]:
set([o['type'] for o in d_b])

{'answer', 'guess', 'question_pick', 'question_pick_with_answer'}

In [109]:
pickle.dump(d_b, open('!final_dataset_6016_with_hints_after_n_step.pkl', 'wb'))

In [114]:
# d_b = pickle.load(open("dataset_17476_from_snapshots_fixed.pkl", 'rb'))
d_b = pickle.load(open("!final_dataset_17476_from_snapshots_fixed.pkl", 'rb'))

In [124]:
print([o for o in d_b if o['type'] == "guess_from_snapshot"][500]['response'])

zambia


In [120]:
set([o['type'] for o in d_b])

{'guess_from_snapshot', 'question_pick_from_snapshot'}

In [153]:
for o in d_b:
    if o['type'] != "guess_from_snapshot":
        continue
    o['prompt'] = o['prompt'].replace("""Format your answer like this: First, reason about your decision for 2-4 sentences. Summarize what you know, summarize space of possibilities, etc.
Then, your last line should be the keyword that you want to guess.
Don't output anything else. Only output the keyword that you want to guess. Don't say anything like "My guess is {keyword}" - always just "{keyword}\"""", 
              """Format your answer like this: only output the keyword that you want to guess, nothing else. Don't say anything like "My guess is {{keyword}}" - always just "{{keyword}}\"""")
    if "sentences" in o['prompt'].lower():
        print("!!!")
    o['response'] = o['response'].split("\n")[-1].strip().replace("Answer: ", "").replace("Is it an ", "").replace("Is it a ", "").replace("Is it in", "").replace("?", "").replace("\"", "")
    # if any symbol is not alphabetical  -output the response
    if any([not c.isalnum() and c != " " and c != "-" and c != "." and c != "," and c != "'" for c in o['response']]):
        print("!", o['response'])

In [164]:
d_b = [o for o in d_b if (o['type'] != "question_pick" and o['type'] != "question_pick_with_answer") or ("?" in o['response'])]
for o in d_b:
    if o['type'] != "questions_candidates_from_snapshot":
        continue
    o['prompt'] = o['prompt'].replace(
    """You need to generate 1 candidates for the next question to ask.
You should mix safe questions with more creative and non-orthodox ones.
Safe questions are usually ones that split the space of possibilities in half.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.
Consider asking non-standard questions like "Does the keyword consist of a single word?, however only do it if you don't know it yet, and if you think that it's important to know at this point in the game.
Consider asking questions like "Does the keyword start with letter 'A'?" or "Does the keyword contain letter 'E'?". However, don't overdo it, focus on more substantial questions.

Format your answer like this: output every question on a new line. Don't output anything else.""",
    """You goal is to choose the best next question.
Safe questions are usually ones that split the space of possibilities in half given the previous answers.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.

You should keep balance of safe questions and more creative and non-orthodox ones.
Safe questions are usually ones that split the space of possibilities in half.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.
Consider asking non-standard questions like "Does the keyword consist of a single word?", however only do it if you don't know it yet, and if you think that it's important to know at this point in the game.
Consider asking questions like "Does the keyword start with letter 'A'?" or "Does the keyword contain letter 'E'?". However, don't overdo it, focus on more substantial questions.

In any case, try to make reason about what decision is the best here, balance risk, and choose the best next question. Act as a competitive extremely smart player that you are.

Don't forget that some inconsistencies in answers can be caused by errors in the answering LLM.

Format your answer like this: Your last and only line should be the question that you want to ask. Just output the question, not "Question: ..." or "I pick question..."
Don't output anything else!"""
)
    if "sentences" in o['prompt']:
        print("!!!")
        break
    o['response'] = o['response'].split("\n")[-1].strip().split(":")[-1]
    if "?" not in o['response'] or ":" in o['response'] or "my" in o['response'].lower() or "i " in o['response'].lower()  or "here " in o['response'].lower():
        print("!", o['response'])
    o['type'] = "question_pick_from_snapshot"  # TODO enable line

In [165]:
pickle.dump(d_b, open('!final_dataset_17476_from_snapshots_fixed.pkl', 'wb'))

In [92]:
d_b = pickle.load(open("!final_dataset_17476_from_snapshots_fixed.pkl", 'rb'))

In [93]:
import pandas as pd

In [94]:
df = pd.DataFrame(d_b)

In [95]:
df['length'] = df['prompt'].apply(len)
df['has_candidates'] = df['prompt'].apply(lambda x: "candidates" in x.lower())

In [96]:
df.has_candidates.sum()

0

In [48]:
df = df.sort_values('length', ascending=False)

In [66]:
print(df.iloc[0]['prompt'])

You are playing "20 Questions" game.
    

    
Here are some candidates for the next question:
Is it a place?
Is it a thing?
Is it a concept?
Is it something you can touch?
Is it used in everyday life?
Is it larger than a car?
Is it found indoors?
Is it related to technology?
Is it made of metal?
Is it a single word?

You goal is to pick the best next question.
Safe questions are usually ones that split the space of possibilities in half given the previous answers.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.

In any case, try to make reason about what decision is the best here, balance risk, and choose the best next question. Act as a competetive extremely smart player that you are.

Don't forget that some inconsistencies in answers can be caused by errors in the answering LLM.

Format your answer like this: First, reason about your decision for 2-4 sentences. Summarize what you know, su

type
answer           2808
question_pick    2807
guess            2765
Name: count, dtype: int64

In [126]:
print(o['prompt'])

You are playing "20 Questions" game.
    

    
Here are some candidates for the next question:
Is it a place?
Is it a thing?
Is it a concept?
Is it something you can touch?
Is it man-made?
Is it used in everyday life?
Is it larger than a car?
Is it found indoors?
Is it electronic?
Is it related to technology?

You goal is to choose the best next question.
Safe questions are usually ones that split the space of possibilities in half given the previous answers.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.

You should keep balance of safe questions and more creative and non-orthodox ones.
Safe questions are usually ones that split the space of possibilities in half.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.
Consider asking non-standard questions like "Does the keyword consist of a single word

In [124]:
print(o['prompt'].index("You goal is to pick the best next question"))

ValueError: substring not found

In [127]:
d_b = pickle.load(open("!final_dataset_11194_basic.pkl", 'rb'))

In [128]:
for o in d_b:
    if o['type'] != "question_pick":
        continue
    to_delete = o['prompt'][o['prompt'].index("Here are some candidates for the next question"):o['prompt'].index("You goal is to choose the best next question")]
    o['prompt'] = o['prompt'].replace(to_delete, "")
    if "candidates" in o['prompt']:
        print("!!!")
        break

In [129]:
for o in d_b:
    o['prompt'] = o['prompt'].replace("You goal", "Your goal")

In [130]:
pickle.dump(d_b, open('!final_dataset_11194_basic.pkl', 'wb'))

In [29]:
d_b = pickle.load(open("!final_dataset_6016_with_hints_after_n_step.pkl", 'rb'))

In [30]:
for o in d_b:
    if o['type'] != "question_pick" and o['type'] != "question_pick_with_answer":
        continue
    to_delete = o['prompt'][o['prompt'].index("Here are some candidates for the next question"):o['prompt'].index("You goal is to pick the best next question")]
    o['prompt'] = o['prompt'].replace(to_delete, "")
    if "candidates" in o['prompt']:
        print("!!!")
        break

In [32]:
for o in d_b:
    o['prompt'] = o['prompt'].replace("You goal", "Your goal")

In [33]:
set([o['type'] for o in d_b])

{'answer', 'guess', 'question_pick', 'question_pick_with_answer'}

In [34]:
pickle.dump(d_b, open('!final_dataset_6016_with_hints_after_n_step.pkl', 'wb'))

In [97]:
d_b = pickle.load(open("!final_dataset_17476_from_snapshots_fixed.pkl", 'rb'))

In [98]:
for o in d_b:
    o['prompt'] = o['prompt'].replace("You goal", "Your goal")

In [99]:
pickle.dump(d_b, open('!final_dataset_17476_from_snapshots_fixed.pkl', 'wb'))

In [73]:
d_b = pickle.load(open("!final_dataset_6016_with_hints_after_n_step.pkl", 'rb'))

In [74]:
# types = set()
for o in d_b:
    if o['type'] != "question_pick":
        continue
    print(o['prompt'])
    # if len(o['response'].split("\n")) > 1:
    #     print(o['response'])
    #     print("!!!!!!!!!!!!")
    #     # types.add(o['type'])
    print("!!!")

You are playing "20 Questions" game.
    

    
Your goal is to pick the best next question.
Safe questions are usually ones that split the space of possibilities in half given the previous answers.
If you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.

In any case, try to make reason about what decision is the best here, balance risk, and choose the best next question. Act as a competetive extremely smart player that you are.

Don't forget that some inconsistencies in answers can be caused by errors in the answering LLM.

Format your answer like this: First, reason about your decision for 2-4 sentences. Summarize what you know, summarize space of possibilities, reason about what question would be the most beneficial.
Then, your last line should be the questions that you've picked.
Don't output anything else.

!!!
You are playing "20 Questions" game.
    

    
Your goal is to pick the best next qu

In [9]:
types

{'answer', 'question_pick', 'question_pick_with_answer'}